In [1]:
"""    
    Indicador: Hectareaje X Hito.
"""

'    \n    Indicador: Hectareaje X Hito.\n'

<h3>Importación Librerías</h3>

In [2]:
import pandas as pd
import os
import seaborn as sns

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime import datetime as df

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias
import funcion_relacion_terrenos_w_ui
import funcion_hectareaje_general_ui

<h3>Parametrización de Variables</h3>

In [3]:
RUTA_TERRENOS_RECONO_TRAMITES_ATENCION_BCGS = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\TERRENOS_RECONOCIMIENTO_TRAMITES_ATENCION_BCGS"
DIRECTORIO_XLSX_2_2_4 = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_4_Hectareaje_X_Hito\zReportes\Seguimiento_Diario"

columnas_terrenos_w_ui = ['codigo','radicado','municipio','proceso','area_ha_cmt12','ID_UI','Zona_UI','Nombre_UI', 'Area_Ha_CMT12', 'Meta_Hito']
columnas_reducion_indicador_1 = ['Hito','Area_Ha_Contractual','hectareaje_visitado_bcgs']

fecha_actual = df.now()
fecha_directorio = str(fecha_actual.strftime("%Y-%m-%d"))

nombre_xlsx_2_2_4 = str(fecha_directorio) + '_indicador_hectareaje_avance_por_hito.xlsx'
RUTA_2_2_4 = os.path.join(DIRECTORIO_XLSX_2_2_4, nombre_xlsx_2_2_4)

ruta_fcTerrenos = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\TERRENO_UNIFICADO_w_GIS"
ruta_bdScrach = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_0_Identificacion_Predial_Total_Ha_Actualizadas\Base_Datos\Terrenos_w_UI.gdb"
ruta_fcUI = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\O_Apoyo_Construccion_UIntervencion\UI_Finales\UI_Finales.gdb\UI_Unificadas\UI_Unificadas"

<h3>Ejecución de Funciones</h3>

In [4]:

# ** Generación de ruta a la relación terreno ui
ruta_tlbTerrenoUI = funcion_relacion_terrenos_w_ui.funcion_terreno_w_ui(ruta_fcTerrenos, ruta_bdScrach, ruta_fcUI)

# ** Genera los DF de los estadísticos generales contractuales de hito vs Ha
df_hitos_ut, df_unidades_intervencion_hitos = funcion_hectareaje_general_ui.funcion_hectareaje_general_hito(ruta_fcUI)

1. Se genera centroides de Terrenos, capa fcTerrenosCentroides, base de datos Terrenos_w_UI.gdb
2. Se genera Join Terrenos Centroides W UI, capa capa_fccentroidesTerrenosWJoinUI, base de datos Terrenos_w_UI.gdb
3. Se borran campos en Terrenos Centroides W UI, capa capa_fccentroidesTerrenosWJoinUI, base de datos Terrenos_w_UI.gdb
4. Se genera Join Terrenos Centroides W Terrenos Originales, capa fcTerrenosUI, base de datos Terrenos_w_UI.gdb
5. Se borran campos en Terrenos W UI, capa fcTerrenosUI, base de datos Terrenos_w_UI.gdb
6.1. Se genera tabla relacion_terreno_ui que relaciona Terrenos con UI, base de datos Terrenos_w_UI.gdb
6.2. Se genera tabla relacion_terreno_ui que relaciona Terrenos con UI, base de datos BD_Consolidada_Lote4.gdb
FINALIZA PROCESO
Se crea dataset de capa UI_Unificadas
Se exporta C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\O_Apoyo_Construccion_UIntervencion\UI_Finales\Estadisticos\indicadores_base_ui_lote_4.xlsx
FINALIZA PROCESO


<h3>Actualización</h3>

In [5]:

# ** Lectura del FC Generado en 2.2.0 {Terrenos Georreferenciados asociados a la Gestión BCGS}
df_terrenosAtencionBCGS = pd.DataFrame.spatial.from_featureclass(RUTA_TERRENOS_RECONO_TRAMITES_ATENCION_BCGS)

# ? Se filtra Actualización
df_terrenosReconocimientoBCGS = df_terrenosAtencionBCGS[df_terrenosAtencionBCGS['proceso']=='Actualización']
# ? Copia de DF
df_terrenosReconocimientoBCGS = df_terrenosReconocimientoBCGS.copy()
# ? Se extrae organizados propio de columnas
df_terrenosReconocimientoBCGS['id_actividad'] = df_terrenosReconocimientoBCGS['estado_tramite'].str[0:1].astype(int)
# ? Se filtra por indicador de proceso
df_terrenosReconocimientoBCGS = df_terrenosReconocimientoBCGS[df_terrenosReconocimientoBCGS['id_actividad'] >= 6]

# ** Lectura del Relación Terreno - UI
df_relTerrenosUI = pd.DataFrame.spatial.from_table(ruta_tlbTerrenoUI)
# ** FC UI Completa
df_UI = pd.DataFrame.spatial.from_featureclass(ruta_fcUI)

# ** Unión, Terrenos con Tabla Relación Terrenos Vs UI
df_terrenosBCGS = pd.merge(left=df_terrenosReconocimientoBCGS,
    right=df_relTerrenosUI,
    left_on="codigo",
    right_on="codigo",
    how="inner"
    )

# ** Unión, Terrenos con ID_UI con FC de UI para completar datos de Hito
df_terrenosBCGS_w_UI = pd.merge(left=df_terrenosBCGS,
    right=df_UI,
    left_on = 'id_ui',
    right_on = 'ID_UI',
    how="inner"
    )

# ** Reducción de Columnas para mejorar análisis
df_terrenosBCGS_w_UI_reducido = df_terrenosBCGS_w_UI[columnas_terrenos_w_ui]
df_terrenosBCGS_w_UI_reducido = df_terrenosBCGS_w_UI_reducido.rename(columns={'Area_Ha_CMT12':'Area_Ha_CTM12_UI', 'area_ha_cmt12':'area_ha_ctm12_predio'})
print(f'{df_terrenosBCGS_w_UI_reducido.shape[0]}')
df_terrenosReconocimientoBCGS

210


,OBJECTID,codigo,codigo_anterior,sig_creador,sig_ultima_edicion,fecha_ultima_edicion,globalid,area_ha_cmt12,radicado,npn,...,vr_step_est,vr_task_nom,vr_task_tip,vr_task_con,vr_task_est,codigo_zona,zona,municipio_estandarizado,SHAPE,id_actividad
222,223,134420000000000020419000000000,,CARLOS_CASTELLANOS,CARLOS_CASTELLANOS,2024-05-29 17:42:55,{D33F19DA-2739-4E81-B1FE-6AE67CBF1862},0.584753,2878,134420000000000020419000000000,...,ATD,Revisión Información Geográfica,FNC,<NA>,<NA>,00,Rural,MariaLaBaja,"{""rings"": [[[4746901.9951, 2657082.293400001],...",8
228,229,134420000000000020785000000000,,CARLOS_CASTELLANOS,CARLOS_CASTELLANOS,2024-05-29 17:42:55,{2424A31E-15CB-4220-B601-7E6F0B897EA1},7.559191,2877,134420000000000020785000000000,...,ATD,Preparar edición geográfica,FNC,<NA>,<NA>,00,Rural,MariaLaBaja,"{""rings"": [[[4743552.5711, 2658977.945800001],...",6
229,230,134420000000000020788000000000,,CARLOS_CASTELLANOS,CARLOS_CASTELLANOS,2024-05-29 17:42:55,{C3BFD54F-2555-40E8-AC93-3F8EFD486A12},8.098875,2877,134420000000000020788000000000,...,ATD,Preparar edición geográfica,FNC,<NA>,<NA>,00,Rural,MariaLaBaja,"{""rings"": [[[4744061.0708, 2657806.2169000003]...",6
230,231,134420000000000020789000000000,,CARLOS_CASTELLANOS,CARLOS_CASTELLANOS,2024-05-29 17:42:55,{1071AA0B-5895-4875-899B-9E4A46D2E23F},11.612215,2877,134420000000000020789000000000,...,ATD,Preparar edición geográfica,FNC,<NA>,<NA>,00,Rural,MariaLaBaja,"{""rings"": [[[4743800.0051, 2658026.445800001],...",6
281,282,134420000000000020207000000000,,CARLOS_CASTELLANOS,CARLOS_CASTELLANOS,2024-05-29 17:42:55,{783D9782-8AB9-4172-98BB-5A693CBEBA5F},13.745559,2878,134420000000000020207000000000,...,ATD,Revisión Información Geográfica,FNC,<NA>,<NA>,00,Rural,MariaLaBaja,"{""rings"": [[[4743513.6002, 2654801.033500001],...",8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4790,4791,134420900000000110001000000000,13442090000110001000,,,NaT,,1.866923,2819,134420900000000110001000000000,...,ATD,Preparar edición geográfica,FNC,<NA>,<NA>,09,Corregimientos Urbanos,MariaLaBaja,"{""rings"": [[[4742549.3452, 2657857.6176999994]...",6
4895,4896,134420900000000090010000000000,13442090000090010000,,,NaT,,0.208175,2817,134420900000000090010000000000,...,ATD,Preparar edición geográfica,FNC,<NA>,<NA>,09,Corregimientos Urbanos,MariaLaBaja,"{""rings"": [[[4742420.8722, 2657828.1384999994]...",6
4930,4931,134421600000000060001000000000,13442160000060001000,,,NaT,,0.052684,2798,134421600000000060001000000000,...,ATD,Preparar edición geográfica,FNC,<NA>,<NA>,16,Corregimientos Urbanos,MariaLaBaja,"{""rings"": [[[4742953.9767, 2655548.4409], [474...",6
4951,4952,134420900000000070006000000000,13442090000070006000,,,NaT,,0.203632,2749,134420900000000070006000000000,...,ATD,Preparar edición geográfica,FNC,<NA>,<NA>,09,Corregimientos Urbanos,MariaLaBaja,"{""rings"": [[[4742289.2458, 2657662.664000001],...",6


<h3>Indicadores</h3>

In [6]:

# ** Indicador 1: Total Gestión BSCG (Activo) por Hito
df_terrenosBCGS_w_UI_indicador_1 = pd.DataFrame(df_terrenosBCGS_w_UI_reducido.groupby(['Meta_Hito'])['area_ha_ctm12_predio'].sum().round(3)).reset_index().rename(columns={'area_ha_ctm12_predio':'hectareaje_visitado_bcgs'})

# ** Indicador 2: Total Gestión BSCG (Activo) por Hito y UI
df_terrenosBCGS_w_UI_indicador_2 = pd.DataFrame(df_terrenosBCGS_w_UI_reducido.groupby(['Meta_Hito','ID_UI','Area_Ha_CTM12_UI'])['area_ha_ctm12_predio'].sum().round(3)).reset_index().rename(columns={'area_ha_ctm12_predio':'hectareaje_visitado_bcgs'})

In [7]:
df_terrenosBCGS_w_UI_indicador_1

,Meta_Hito,hectareaje_visitado_bcgs
0,Hito 2,1617.514
1,Hito 3,138.844
2,Hito 4,765.156


In [8]:
df_indicador_1_contrato = pd.merge(left=df_hitos_ut,
    right=df_terrenosBCGS_w_UI_indicador_1,
    left_on = 'Hito',
    right_on = 'Meta_Hito',
    how="inner")

df_indicador_1_contrato_reducido = df_indicador_1_contrato[columnas_reducion_indicador_1]

df_indicador_1_contrato_reducido = df_indicador_1_contrato_reducido.copy()

df_indicador_1_contrato_reducido['porcentaje_visitado_bcgs'] = ((df_indicador_1_contrato_reducido['hectareaje_visitado_bcgs'] / df_indicador_1_contrato_reducido['Area_Ha_Contractual']) * 100).round(4)
df_indicador_1_contrato_reducido['hectareaje_predios_finalizados'] = 0

df_indicador_1_contrato_reducido

,Hito,Area_Ha_Contractual,hectareaje_visitado_bcgs,porcentaje_visitado_bcgs,hectareaje_predios_finalizados
0,Hito 2,16243.92,1617.514,9.9577,0
1,Hito 3,54146.40,138.844,0.2564,0
2,Hito 4,81219.60,765.156,0.9421,0


In [9]:
df_terrenosBCGS_w_UI_indicador_2['Area_Ha_CTM12_UI'] = df_terrenosBCGS_w_UI_indicador_2['Area_Ha_CTM12_UI'].round(3)
df_terrenosBCGS_w_UI_indicador_2['porcentaje_visitado_bcgs'] = ((df_terrenosBCGS_w_UI_indicador_2['hectareaje_visitado_bcgs'] / df_terrenosBCGS_w_UI_indicador_2['Area_Ha_CTM12_UI']) * 100).round(4)
df_terrenosBCGS_w_UI_indicador_2['hectareaje_predios_finalizados'] = 0

df_terrenosBCGS_w_UI_indicador_2

,Meta_Hito,ID_UI,Area_Ha_CTM12_UI,hectareaje_visitado_bcgs,porcentaje_visitado_bcgs,hectareaje_predios_finalizados
0,Hito 2,ZI_MB_11,4793.979,736.277,15.3584,0
1,Hito 2,ZI_MB_14,898.76,190.489,21.1946,0
2,Hito 2,ZI_MB_16,2153.949,690.749,32.069,0
3,Hito 3,ZI_MB_15_15,46.72,1.285,2.7504,0
4,Hito 3,ZI_MB_15_4,4.828,0.214,4.4325,0
5,Hito 3,ZI_MB_15_7,35.778,7.956,22.2371,0
6,Hito 3,ZI_MB_17_0,194.065,0.495,0.2551,0
7,Hito 3,ZI_RP_03,2189.123,128.894,5.8879,0
8,Hito 4,ZI_RP_02,3190.951,721.522,22.6115,0
9,Hito 4,ZI_RP_04,2894.664,43.635,1.5074,0


<h3>Generación XLSX</h3>

In [10]:
with pd.ExcelWriter(RUTA_2_2_4, engine='xlsxwriter') as writer:
    df_indicador_1_contrato_reducido.to_excel(writer, sheet_name='Indicador_1', index=False)
    df_terrenosBCGS_w_UI_indicador_2.to_excel(writer, sheet_name='Indicador_2', index=False)

print("Finaliza la exportación del XLSX")

Finaliza la exportación del XLSX
